In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import torch.utils.data as utils
import librosa
import soundfile as sf
import time
import os

In [2]:
sampleSize=150000
sample_rate=16000
quantization_channels=256
dilations=[2**i for i in range(9)]*3
skipDim=512
residualDim=32
filterSize=3

In [3]:
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [4]:
use_cuda = torch.cuda.is_available()
torch.manual_seed(1)
device = torch.device("cuda" if use_cuda else "cpu")
#device = 'cpu'
torch.set_default_tensor_type('torch.cuda.FloatTensor')
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [5]:
def mu_law_encode(audio, quantization_channels=quantization_channels):
    '''Quantizes waveform amplitudes.'''
    mu = (quantization_channels - 1)*1.0
    # Perform mu-law companding transformation (ITU-T, 1988).
    # Minimum operation is here to deal with rare large amplitudes caused
    # by resampling.
    safe_audio_abs = np.minimum(np.abs(audio), 1.0)
    magnitude = np.log1p(mu * safe_audio_abs) / np.log1p(mu)
    signal = np.sign(audio) * magnitude
    # Quantize signal to the specified number of levels.
    #if(forX):return signal
    return ((signal + 1) / 2 * mu + 0.5).astype(int)
def mu_law_decode(output, quantization_channels=quantization_channels):
    '''Recovers waveform from quantized values.'''
    mu = quantization_channels - 1
    # Map values back to [-1, 1].
    signal = 2 * ((output*1.0) / mu) - 1
    # Perform inverse of mu-law transformation.
    magnitude = (1 / mu) * ((1 + mu)**np.abs(signal) - 1)
    return np.sign(signal) * magnitude

In [6]:
def readAudio(name):
    audio0, samplerate = sf.read(name, dtype='float32')
    return librosa.resample(audio0.T, samplerate, sample_rate).reshape(-1)
p=['./vsCorpus/origin_mix.wav','./vsCorpus/origin_vocal.wav',
   './vsCorpus/origin_mix.wav','./vsCorpus/origin_vocal.wav','./vsCorpus/pred_mix.wav']
xtrain,ytrain,xval,yval,xtest=readAudio(p[0]),readAudio(p[1]),readAudio(p[2]),readAudio(p[3]),readAudio(p[4])
assert((xtrain==xval).all())
assert((ytrain==yval).all())
assert((xtrain != ytrain).any())
#xtrain,ytrain,xval,yval=xtrain[:-sampleSize],ytrain[:-sampleSize],xval[-sampleSize:],yval[-sampleSize:]
xtrain,ytrain,xval,yval=xtrain[:-sampleSize],ytrain[:-sampleSize],xval[:sampleSize],yval[:sampleSize]
xtrain,xval,xtest=xtrain.reshape(1,1,-1),xval.reshape(1,1,-1),xtest.reshape(1,1,-1)
ytrain,yval=ytrain.reshape(1,-1),yval.reshape(1,-1)

In [7]:
xmean,xstd = xtrain.mean(),xtrain.std()
xtrain=(xtrain-xmean)/xstd
xval=(xval-xmean)/xstd
xtest=(xtest-xmean)/xstd
ytrain,yval=mu_law_encode(ytrain),mu_law_encode(yval)

In [8]:
xtrain,ytrain,xval,yval,xtest = torch.from_numpy(xtrain).type(torch.float32),\
                                torch.from_numpy(ytrain).type(torch.LongTensor),\
                                torch.from_numpy(xval).type(torch.float32),\
                                torch.from_numpy(yval).type(torch.LongTensor),\
                                torch.from_numpy(xtest).type(torch.float32)

In [9]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        sd,qd,rd = skipDim,quantization_channels,residualDim
        self.causal = nn.Conv1d(in_channels=1,out_channels=rd,kernel_size=3,padding=1)
        self.layer=dict()
        for i, d in enumerate(dilations):
            self.layer['tanh'+str(i)] = nn.Conv1d(in_channels=rd,out_channels=rd,kernel_size=3,padding=d,dilation=d)
            self.layer['sigmoid'+str(i)] = nn.Conv1d(in_channels=rd,out_channels=rd,kernel_size=3,padding=d,dilation=d)
            self.layer['skip'+str(i)] = nn.Conv1d(in_channels=rd,out_channels=sd,kernel_size=1,padding=0)
            self.layer['dense'+str(i)] = nn.Conv1d(in_channels=rd,out_channels=rd,kernel_size=1,padding=0)
        self.post1 = nn.Conv1d(in_channels=sd,out_channels=sd,kernel_size=1,padding=0)
        self.post2 = nn.Conv1d(in_channels=sd,out_channels=qd,kernel_size=1,padding=0)
        self.tanh,self.sigmoid = nn.Tanh(),nn.Sigmoid()

    def forward(self, x):
        x = self.causal(x)
        #print('x.shape',x.shape)
        skip_connections = torch.zeros([1,skipDim,x.shape[2]],dtype=torch.float32,device=device)
        #skip_connections = []
        #print('skip_connections',skip_connections.shape)
        for i, dilation in enumerate(dilations):
            xinput = x.clone()
            x1 = self.tanh(self.layer['tanh'+str(i)](x))
            #print('tanh.shape',x1.shape)
            x2 = self.sigmoid(self.layer['sigmoid'+str(i)](x))
            #print('sigmoid.shape',x2.shape)
            x = x1*x2
            #print('multi',x3.shape)
            skip_connections += self.layer['skip'+str(i)](x)
            #print(skip_connections.cpu()[:,:,:10])
            #cur =self.layer['skip'+str(i)](x3)
            #skip_connections.append(cur)
            #print('skip.shape',self.layer['skip'+str(i)](x).shape)
            x = self.layer['dense'+str(i)](x)
            #print('dense.shape',x.shape)
            x += xinput
        #skip_connections = torch.cat(skip_connections,dim=0)
        #skip_connections = torch.sum(skip_connections,dim=0,keepdim=True)
        x = self.post2(F.relu(self.post1(F.relu(skip_connections))))
        return x

model = Net().cuda()
criterion = nn.CrossEntropyLoss().cuda()
#optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
optimizer = optim.Adam(model.parameters(),weight_decay=1e-5)

In [10]:
#torch.sum(c,dim=0,keepdim=True)

In [11]:
#model = torch.load('torchmodel0.681600')

In [15]:
def val():
    model.eval()
    startval_time = time.time()
    with torch.no_grad():
        data, target = xval.to(device), yval.to(device)
        output = model(data)
        #print(output.shape)
        #print(output[:,:,:10])
        val_loss = criterion(output, target).item()
    print('\nval set:  {:.4f}, ({:.3f} sec/step)\n'.format(val_loss,time.time()-startval_time))

def test():
    model.eval()
    startval_time = time.time()
    with torch.no_grad():
        output = model(xtest.to(device))
        pred = output.max(1, keepdim=True)[1].cpu().numpy().reshape(-1)
        ans = mu_law_decode(pred)
        sf.write('./vsCorpus/resultxte.wav', ans, sample_rate)
        
        output = model(xtrain[:,:,:sampleSize].to(device))
        pred = output.max(1, keepdim=True)[1].cpu().numpy().reshape(-1)
        ans = mu_law_decode(pred)
        sf.write('./vsCorpus/resultxtr.wav', ans, sample_rate)
        print('stored done\n')
    
def train(epoch):
    model.train()
    #idx = np.arange(xtrain.shape[-1] - 2 * sampleSize)
    #np.random.shuffle(idx)
    idx = np.array([0]*100000)
    for i, ind in enumerate(idx):
        start_time = time.time()
        data, target = xtrain[:,:,ind:ind+sampleSize].to(device), ytrain[:,ind:ind+sampleSize].to(device)
        output = model(data)
        #print(output.shape)
        #print(output[:,:,:10])
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)] Loss: {:.6f} , ({:.3f} sec/step)'.format(
                epoch, i, len(idx),100. * i / len(idx), loss.item(),time.time() - start_time))
        if i % 100 == 0:
            val()
            test()
            torch.save(model, 'loss3.94')

In [ ]:
for epoch in range(1000):
    train(epoch)
    test()

Train Epoch: 0 [0/100000 (0%)] Loss: 3.893718 , (1.763 sec/step)

val set:  3.8925, (0.680 sec/step)

stored done



/home/coder.chenshicheng/anaconda3/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Train Epoch: 0 [1/100000 (0%)] Loss: 3.892546 , (1.762 sec/step)
Train Epoch: 0 [2/100000 (0%)] Loss: 3.891579 , (1.762 sec/step)
Train Epoch: 0 [3/100000 (0%)] Loss: 3.890687 , (1.762 sec/step)
Train Epoch: 0 [4/100000 (0%)] Loss: 3.889650 , (1.763 sec/step)
Train Epoch: 0 [5/100000 (0%)] Loss: 3.888569 , (1.761 sec/step)
Train Epoch: 0 [6/100000 (0%)] Loss: 3.887532 , (1.762 sec/step)
Train Epoch: 0 [7/100000 (0%)] Loss: 3.886544 , (1.763 sec/step)
Train Epoch: 0 [8/100000 (0%)] Loss: 3.885559 , (1.761 sec/step)
Train Epoch: 0 [9/100000 (0%)] Loss: 3.884556 , (1.761 sec/step)
Train Epoch: 0 [10/100000 (0%)] Loss: 3.883617 , (1.763 sec/step)
Train Epoch: 0 [11/100000 (0%)] Loss: 3.882710 , (1.762 sec/step)
Train Epoch: 0 [12/100000 (0%)] Loss: 3.881768 , (1.763 sec/step)
Train Epoch: 0 [13/100000 (0%)] Loss: 3.880730 , (1.763 sec/step)
Train Epoch: 0 [14/100000 (0%)] Loss: 3.879642 , (1.763 sec/step)
Train Epoch: 0 [15/100000 (0%)] Loss: 3.878591 , (1.761 sec/step)
Train Epoch: 0 [16/

Train Epoch: 0 [125/100000 (0%)] Loss: 3.781085 , (1.763 sec/step)
Train Epoch: 0 [126/100000 (0%)] Loss: 3.779909 , (1.763 sec/step)
Train Epoch: 0 [127/100000 (0%)] Loss: 3.777995 , (1.763 sec/step)
Train Epoch: 0 [128/100000 (0%)] Loss: 3.775819 , (1.764 sec/step)
Train Epoch: 0 [129/100000 (0%)] Loss: 3.774038 , (1.764 sec/step)
Train Epoch: 0 [130/100000 (0%)] Loss: 3.773095 , (1.761 sec/step)
Train Epoch: 0 [131/100000 (0%)] Loss: 3.772969 , (1.761 sec/step)
Train Epoch: 0 [132/100000 (0%)] Loss: 3.773357 , (1.764 sec/step)
Train Epoch: 0 [133/100000 (0%)] Loss: 3.773412 , (1.763 sec/step)
Train Epoch: 0 [134/100000 (0%)] Loss: 3.772871 , (1.763 sec/step)
Train Epoch: 0 [135/100000 (0%)] Loss: 3.771157 , (1.762 sec/step)
Train Epoch: 0 [136/100000 (0%)] Loss: 3.769049 , (1.765 sec/step)
Train Epoch: 0 [137/100000 (0%)] Loss: 3.766967 , (1.763 sec/step)
Train Epoch: 0 [138/100000 (0%)] Loss: 3.765624 , (1.761 sec/step)
Train Epoch: 0 [139/100000 (0%)] Loss: 3.764968 , (1.765 sec/s

Train Epoch: 0 [247/100000 (0%)] Loss: 3.684542 , (1.763 sec/step)
Train Epoch: 0 [248/100000 (0%)] Loss: 3.684036 , (1.766 sec/step)
Train Epoch: 0 [249/100000 (0%)] Loss: 3.681840 , (1.765 sec/step)
Train Epoch: 0 [250/100000 (0%)] Loss: 3.679036 , (1.762 sec/step)
Train Epoch: 0 [251/100000 (0%)] Loss: 3.676687 , (1.763 sec/step)
Train Epoch: 0 [252/100000 (0%)] Loss: 3.675671 , (1.763 sec/step)
Train Epoch: 0 [253/100000 (0%)] Loss: 3.676225 , (1.761 sec/step)
Train Epoch: 0 [254/100000 (0%)] Loss: 3.677595 , (1.765 sec/step)
Train Epoch: 0 [255/100000 (0%)] Loss: 3.677567 , (1.765 sec/step)
Train Epoch: 0 [256/100000 (0%)] Loss: 3.675839 , (1.764 sec/step)
Train Epoch: 0 [257/100000 (0%)] Loss: 3.672422 , (1.763 sec/step)
Train Epoch: 0 [258/100000 (0%)] Loss: 3.669604 , (1.766 sec/step)
Train Epoch: 0 [259/100000 (0%)] Loss: 3.668543 , (1.767 sec/step)
Train Epoch: 0 [260/100000 (0%)] Loss: 3.668927 , (1.766 sec/step)
Train Epoch: 0 [261/100000 (0%)] Loss: 3.669385 , (1.765 sec/s

Train Epoch: 0 [369/100000 (0%)] Loss: 3.597081 , (1.763 sec/step)
Train Epoch: 0 [370/100000 (0%)] Loss: 3.595498 , (1.766 sec/step)
Train Epoch: 0 [371/100000 (0%)] Loss: 3.594265 , (1.762 sec/step)
Train Epoch: 0 [372/100000 (0%)] Loss: 3.593440 , (1.764 sec/step)
Train Epoch: 0 [373/100000 (0%)] Loss: 3.593018 , (1.764 sec/step)
Train Epoch: 0 [374/100000 (0%)] Loss: 3.592776 , (1.764 sec/step)
Train Epoch: 0 [375/100000 (0%)] Loss: 3.592682 , (1.763 sec/step)
Train Epoch: 0 [376/100000 (0%)] Loss: 3.592549 , (1.763 sec/step)
Train Epoch: 0 [377/100000 (0%)] Loss: 3.592453 , (1.764 sec/step)
Train Epoch: 0 [378/100000 (0%)] Loss: 3.591959 , (1.765 sec/step)
Train Epoch: 0 [379/100000 (0%)] Loss: 3.591687 , (1.763 sec/step)
Train Epoch: 0 [380/100000 (0%)] Loss: 3.591460 , (1.766 sec/step)
Train Epoch: 0 [381/100000 (0%)] Loss: 3.591098 , (1.764 sec/step)
Train Epoch: 0 [382/100000 (0%)] Loss: 3.590726 , (1.764 sec/step)
Train Epoch: 0 [383/100000 (0%)] Loss: 3.589360 , (1.764 sec/s

In [ ]:
#model = torch.load('torchmodel')